In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate #1 prompt from string, 2 prompt from chat
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector




chat = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )
examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variable):
        from random import choice
        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="What do you know about {country}?",
    input_variables=["country"]
)



chain = prompt|chat

prompt.format(country="brazil")

'Human:What do you know about Greece?\nAI:\nI know this:\nCapital: Athens\nLanguage: Greek\nFood: Souvlaki and Feta Cheese\nCurrency: Euro\n\n\nWhat do you know about brazil?'

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate #1 prompt from string, 2 prompt from chat
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
chat = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
    )
examples = [
{
"country": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"country": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"country": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human","What do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ('system','you are geography expert.'),
    example_prompt,
    ('human', 'what do you know about {country}?')
])

chain = prompt|chat

chain.invoke({
    "country":"vietnam"
})

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate #1 prompt from string, 2 prompt from chat
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt
from langchain.prompts.pipeline import PipelinePromptTemplate

prompt = load_prompt("./prompt.json")

chat = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
        ]
)
intro = PromptTemplate.from_template(
"""
You are a role playing assistant.
And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
"""
This is an example of how you talk:

Human: {example_question}
You: {example_answer}
"""
)

start = PromptTemplate.from_template(
"""
Start now!

Human: {question}
You:
"""
)

final = PromptTemplate.from_template(
"""
{intro}

{example}

{start}
"""
)

prompts = [
    ("intro"=intro),
    ("example",example),
    ("start",start)
]

full_prompt = PipelinePromptTemplate(final_prompt=final,pipeline_prompts=prompts)

full_prompt.format(
    character="pirate"
    example_question= "what is your location",
    example_answer = "Arrrg! That is a secret!! arg eg",
    question= " What is your favorite food?"
)

'what is the capital of Germany'